# Shapes

## Basic Array

In [ ]:
class Arr:
    def __init__(self, data):
        data = [item for item in data]
        self.dtype = dtype(data)
        self.data = [self.dtype(item) for item in data]

    def __str__(self):
        return (f"Arr: {self.data}")

    __repr__ = __str__

    def __len__(self):
        return len(self.data)

    def __iter__(self):
        return iter(self.data)

    ### Math

    def __add__(self, other):
        return zip_apply(self, other, self.dtype.__add__)

    def __sub__(self, other):
        return zip_apply(self, other, self.dtype.__sub__)

    def __mul__(self, other):
        return zip_apply(self, other, self.dtype.__mul__)

    def __truediv__(self, other):
        return zip_apply(self, other, self.dtype.__truediv__)

    def __abs__(self):
        return Arr(map(self.dtype.__abs__, self.data))


def zip_apply(left, right, f):
    # Length is the same
    assert len(left) == len(right), f'arrays are not of same shape'
    # Type is the same
    assert dtype(left) == dtype(right), f'Arrays are not of same dtype'
    # We can do the work
    result = [f(l, r) for (l, r) in zip(left, right)]
    return Arr(result)


def dtype(obj):
    """Returns the dtype of the array"""
    dtype = int
    for item in obj:
        itype = type(item)
        if itype == str:  # str is the largest, so dtype is str
            return str
        if itype == float:  # We haven't seen str by now so type is either float or int
            dtype = float
    return dtype

# Shape Method
Behind the scenes, the array is not "aware" of it having a shape, or that it has dimensions. Shape and dimensions are metadata which we use when we do operations.

In [ ]:
class Arr(Arr):
    def __init__(self, data):
        data = [item for item in data]
        self.dtype = dtype(data)
        self.data = [self.dtype(item) for item in data]
        ### New Code
        self.size = len(self.data) # Because len will be something else
        self.shape = (self.size,)
        self.ndim = len(self.shape)

In [ ]:
arr = Arr(range(16))
print(arr.shape)
print(arr.ndim)
print(arr)

In [ ]:
arr.shape = (4,4)
arr.ndim = 2
print (arr)

Not what we expected

## Better print for arrays

In [ ]:
def print_array(arr):
    ret = "Arr: \n" # We will add more strings to this "main" string
    if arr.ndim < 2: 
        ret += str(arr.data)
    else:
        rows, cols = arr.shape
        for i, val in enumerate(arr.data):
            ret += str(val).center(6) # str.center - check it out.
            if i % cols == cols -1: # in "real" math this means i mod cols = -1
                ret += '\n'
    
    return ret

In [ ]:
arr = Arr(range(16))
print(print_array(arr))

In [ ]:
arr.shape = (4,4)
arr.ndim = 2
print(print_array(arr))

**IT WORKS** Let's make this official.

In [ ]:
Arr.__str__ = print_array
Arr.__repr__ = print_array

In [ ]:
arr

# Reshape
Setting the `shape` is dangerous. It requires also changing the `ndim` and does check if the new shape makes sense

## (Bad) Examples

In [ ]:
arr = Arr(range(16))
arr.shape = (4,4)
arr

In [ ]:
arr = Arr(range(16))
arr.ndim = 2
try:
    print (arr)
except Exception as e:
    print(e)

In [ ]:
arr = Arr(range(16))
arr.ndim = 2
arr.shape = (-3,4)
arr

## `Arr.reshape`
Requirements:  
* Change shape and ndim always together.
* Make sure rows * cols = size
* Have the option to change to either 1 dim or 2.
    * example: `arr.reshape(4,4)` or `arr.reshape(16)` both acceptable
* BONUS: have an option set one of the dimensions as -1 and have the array do the math.

In [ ]:
16 // -1

In [ ]:
# Ex
class Arr(Arr):
    def reshape(self, rows, cols=None):
        ret = Arr(self.data)
        pass # Your Code Here
        ret.shape = (rows, cols)
        ret.ndim = len(ret.shape)
        return ret
        

In [ ]:
arr.reshape(4,4)

In [ ]:
arr.reshape(2,-1)

In [ ]:
arr.reshape(-1,2)

In [ ]:
arr.reshape(5,3) # Error

In [ ]:
arr.reshape(-1,7) # Error

## fix `len`
The `len` of an array (in numpy) is defined as the size of the first dimension.

In [ ]:
class Arr(Arr):
    def __len__(self):
        return self.shape[0]

# Slicing with `__getitem__`
We want to be able to [slice](Part%209%20-%20Minimals.ipynb#Get-Item) an array. We will implement `arr[n:m]` or `arr[n1:m1,n2:m2]`

In [ ]:
b = arr.reshape(4,4)
b[3]

## The Easy Part
If the array is 1-dim, it works as a list

In [ ]:
def arr_getter(arr, items):
    if arr.ndim == 1:
        return arr.data[items]
    if arr.ndim == 2:
        raise Exception('not supported')

In [ ]:
arr_getter(arr, slice(3,8))

In [ ]:
arr_getter(b, slice(3,8))

In [ ]:
Arr.__getitem__ = lambda self, items: arr_getter(arr, items)
Arr(range(16))[4:10]

## 2 Dimensions
Now this becomes complicated. We get 2 slices. For each slice we have `start,stop,step`.  
Algorithm:
1. Find the first index corresponding to coordinates (row_start, col_start)
1. Move to next index according to correct step.
1. Finish when we reached correct size.

### Some helper functions
* Move between coordinates to index and back.
* Get `len` of new `arr.data` by the slices.

In [ ]:
def indices2len(start, stop, step):
    if step > 0:
        return (stop - start - 1) // step + 1
    elif step == 0:
        return 1
    else:
        # No reversing
        raise ValueError
        

def coord2idx(arr, row, col):
    return row * arr.shape[1] + col

def idx2coord(arr, idx):
    row = idx // arr.shape[1]
    col = idx % arr.shape[0]
    return row, col

In [ ]:
def arr_getter(arr, items):
    if not isinstance(items, tuple):  # got only one slicer
        items = (items,)
    assert arr.ndim >= len(items), f'More slicers ({len(items)}) than dimensions ({arr.ndim})'
    if arr.ndim == 1:
        return arr.data[items[0]]
    if arr.ndim == 2:
        r_start, r_stop, r_step = items[0].indices(arr.shape[0])
        c_start, c_stop, c_step = items[1].indices(arr.shape[1])
        new_shape = (indices2len(r_start, r_stop, r_step), indices2len(c_start, c_stop, c_step))
        return array_iterator(arr, new_shape, r_start, r_step, r_stop, c_step, c_stop, c_start)

In [ ]:
def array_iterator(arr, new_shape, r_start, r_step, r_stop, c_step, c_stop, c_start):
    r_index, c_index = r_start, c_start # index is set to start
    ret = [] # We will populate this
    for _ in range(new_shape[0] * new_shape[1]):
        if c_index >= c_stop:
            # We reached end of line
            c_index = c_start # Go back to start of line which is first coloumn
            r_index += r_step # Advance the row
        if r_index >= r_stop:
            # We reached the end
            break
        _index = coord2idx(arr, r_index, c_index) # Which index to take from the data
        try:
            ret.append(arr.data[_index])
        except IndexError:
            break
        c_index += c_step
    ret = Arr(ret).reshape(*new_shape)
    return ret

In [ ]:
arr_getter(b,(slice(None),slice(None)))

In [ ]:
arr_getter(b,(slice(1,None),slice(None,None,2)))

In [ ]:
del(arr)

In [ ]:
Arr.__getitem__ = arr_getter
b = Arr(range(16)).reshape(4,4)
b[1:,::2]

## Exercise
The following will not work. Fix them.

In [ ]:
b[1:] # If only one index is given, it means rows.

In [ ]:
b[1,2] # Not familiar with ints

In [ ]:
(2,3,4) + (5,)

In [ ]:
# Ex
def arr_getter(arr, items):
    if not isinstance(items, tuple):  # got only one slicer
        items = (items,)
    assert arr.ndim >= len(items), f'More slicers ({len(items)}) than dimensions ({arr.ndim})'
    if arr.ndim == 1:
        return arr.data[items[0]]
    if arr.ndim == 2:
        # Less Then 2 Items
        pass # Your Code Here
        # Int location
        pass # Your Code Here
        r_start, r_stop, r_step = items[0].indices(arr.shape[0])
        c_start, c_stop, c_step = items[1].indices(arr.shape[1])
        new_shape = (indices2len(r_start, r_stop, r_step), indices2len(c_start, c_stop, c_step))
        return array_iterator(arr, new_shape, r_start, r_step, r_stop, c_step, c_stop, c_start)

In [ ]:
arr_getter(b,(slice(1,3)))

In [ ]:
arr_getter(b,(1,3))

In [ ]:
Arr.__getitem__ = arr_getter
b = Arr(range(16)).reshape(4,4)
print (b[1:,::2])
print (b[1,3])
print (b[:,1])